<h1 id="Project:-Euro-Soccer-Database-Analysis">Project: Euro Soccer Database Analysis<a class="anchor-link" href="#Project:-Euro-Soccer-Database-Analysis">¶</a></h1><h2 id="Table-of-Contents">Table of Contents<a class="anchor-link" href="#Table-of-Contents">¶</a></h2><ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>


<p><a id="intro"></a></p>
<h2 id="Introduction">Introduction<a class="anchor-link" href="#Introduction">¶</a></h2><p>As a Soccer fan, I have decided to take up the Euro Soccer Database for analysis. I have used DB Browser for SQLite to extract the datasets from the Database. Following are the questions that I have come up for analysis. I have divided the questions or research analysis set into two groups, one related to Players and one to Teams:</p><p></p><p>On Players:</p>
<ol>
<li>Which player has the most penalties? </li>
<li>Attributes of better players than average?</li>
<li>Age and Overall Rating relationship analysis.</li>
</ol>
<p></p><p>On Teams:</p>
<ol>
<li>Which team improved the most over the time period? </li>
<li>Home and Away Games - winning and losing ratio analysis.</li>
</ol>


<h3 id="SQL-Queries-Used:">SQL Queries Used:<a class="anchor-link" href="#SQL-Queries-Used:">¶</a></h3><p>Following SQL Queries are used to combine the tables mentioned in the section above.</p>
<ol>
<li><p>For players dataset: I needed the player's name and birthday from Player table and the attributes from Player_Attributes table.</p>
<div class="highlight"><pre><span></span><span class="k">SELECT</span> <span class="n">p</span><span class="p">.</span><span class="n">player_name</span><span class="p">,</span> <span class="n">p</span><span class="p">.</span><span class="n">birthday</span><span class="p">,</span> <span class="n">pa</span><span class="p">.</span><span class="o">*</span> <span class="k">FROM</span> <span class="n">Player</span> <span class="n">p</span> <span class="k">JOIN</span> <span class="n">Player_Attributes</span> <span class="n">pa</span> <span class="k">ON</span> <span class="n">p</span><span class="p">.</span><span class="n">player_api_id</span> <span class="o">=</span> <span class="n">pa</span><span class="p">.</span><span class="n">player_api_id</span><span class="p">;</span>
</pre></div>
</li>
<li><p>To form team_match_records dataset: I needed the date, match_api_id, home_team_api_id, away_team_api_id, home_team_goal and away_team_goal from the Match Table.</p>
<div class="highlight"><pre><span></span><span class="k">SELECT</span> <span class="kt">date</span><span class="p">,</span> <span class="n">match_api_id</span><span class="p">,</span> <span class="n">home_team_api_id</span><span class="p">,</span> <span class="n">away_team_api_id</span><span class="p">,</span> <span class="n">home_team_goal</span><span class="p">,</span> <span class="n">away_team_goal</span> <span class="k">FROM</span> <span class="k">Match</span><span class="p">;</span>
</pre></div>
</li>
</ol>


<h3 id="Importing-Packages">Importing Packages<a class="anchor-link" href="#Importing-Packages">¶</a></h3>

In [2]:

# importing the libraries needed for this project
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
%matplotlib inline

<p><a id="wrangling"></a></p>
<h2 id="Data-Cleaning-and-Wrangling---Players">Data Cleaning and Wrangling - Players<a class="anchor-link" href="#Data-Cleaning-and-Wrangling---Players">¶</a></h2>


In [4]:
# Load the players dataset
df_player_records = pd.read_csv('players.csv')



<h3 id="Perform-Data-Check-and-Inspections">Perform Data Check and Inspections<a class="anchor-link" href="#Perform-Data-Check-and-Inspections">¶</a></h3>


In [7]:
# Checking the overall data information and types of the data
df_player_records.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183978 entries, 0 to 183977
Data columns (total 44 columns):
player_name            183978 non-null object
birthday               183978 non-null object
id                     183978 non-null int64
player_fifa_api_id     183978 non-null int64
player_api_id          183978 non-null int64
date                   183978 non-null object
overall_rating         183142 non-null float64
potential              183142 non-null float64
preferred_foot         183142 non-null object
attacking_work_rate    180748 non-null object
defensive_work_rate    183142 non-null object
crossing               183142 non-null float64
finishing              183142 non-null float64
heading_accuracy       183142 non-null float64
short_passing          183142 non-null float64
volleys                181265 non-null float64
dribbling              183142 non-null float64
curve                  181265 non-null float64
free_kick_accuracy     183142 non-null float64
long_pas

In [8]:
# check the dimensions of the dataset
df_player_records.shape

(183978, 44)

In [9]:

# check for duplication of data
df_player_records.duplicated().sum()


0

<p>For my analysis on this data, I'm interested in the columns player_api_id, player_name, birthday, date, overall_rating, penalties and the goalkeeper attributes. 
So, I'll create a new data frame with just these columns.</p>

In [10]:

df_players = df_player_records[['player_api_id','player_name','birthday','date','overall_rating','penalties','gk_diving','gk_handling','gk_kicking','gk_positioning','gk_reflexes']]



In [11]:
# check if the created dataframe is correct
df_players.head(2)


,player_api_id,player_name,birthday,date,overall_rating,penalties,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,505942,Aaron Appindangoye,1992-02-29 00:00:00,2016-02-18 00:00:00,67.0,48.0,6.0,11.0,10.0,8.0,8.0
1,505942,Aaron Appindangoye,1992-02-29 00:00:00,2015-11-19 00:00:00,67.0,48.0,6.0,11.0,10.0,8.0,8.0


In [12]:
# check for null values
df_players.isnull().sum()

player_api_id       0
player_name         0
birthday            0
date                0
overall_rating    836
penalties         836
gk_diving         836
gk_handling       836
gk_kicking        836
gk_positioning    836
gk_reflexes       836
dtype: int64

<p>Since, the datatypes for the missig values are all float types, I'll fill the null values with 0 as replacing with 0 will not impact my calculations.</p>

In [13]:
# fill null values with 0
df_players.fillna(0, inplace = True)

C:\Users\cr174k\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


<h3 id="Quick-Notes-on-Data-Cleaning-and-Data-Wrangling.">Quick Notes on Data Cleaning and Data Wrangling.<a class="anchor-link" href="#Quick-Notes-on-Data-Cleaning-and-Data-Wrangling.">¶</a></h3><ol>
<li>For this dataset players, there are no duplicate records.</li>
<li>In total there are 183978 rows with 44 columns.</li>
<li>Removed those columns which will not be used for my analysis.</li>
<li>Filled the missing values with 0 as the data types of the missing values which I'm interested are of float data types.</li>
</ol>

<p><a id="eda"></a></p>
<h2 id="Exploratory-Data-Analysis">Exploratory Data Analysis<a class="anchor-link" href="#Exploratory-Data-Analysis">¶</a></h2><h3 id="1.-Which-player-had-the-most-penalties?">1. Which player had the most penalties?<a class="anchor-link" href="#1.-Which-player-had-the-most-penalties?">¶</a></h3>